# Deep Learning Graduate Course Competition

In [1]:
# !unzip data.zip

**Importing Required Libraries and Functions**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd /content/drive/My Drive/DL-Comp

/content/drive/My Drive/DL-Comp


In [1]:
# External Libraries
import torch
import torch.optim as optim
import torchvision.transforms as transforms

# Internal Helper ibraries/Functions
from resnet import ResNet
from utils import get_data_loaders, train, test, save_checkpoint, load_checkpoint

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_PATH = "data"

**Initializing Our Model (implementation [here](resnet.py))**

In [2]:
model = ResNet(num_blocks=[4,4,3]).to(DEVICE)


**Number of Parameters in our Model**

In [5]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 4662926


**Transforms**

In [6]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

**Setting Up Data Loaders**

In [7]:
labels_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
train_loader, test_loader = get_data_loaders(dataset_path=DATA_PATH,batch_size=128,test_transform=test_transform, train_transform=train_transform)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


**Training Parameters**

In [8]:
epochs = 200

loss_function = torch.nn.CrossEntropyLoss()

weight_decay = 0.0005
lr = 0.1
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

**Loading Checkpoint**

In [9]:
starting_epoch = 0
# starting_epoch = load_checkpoint(model, optimizer, "checkpoints/my_checkpoint.pth.tar") + 1

**Training**

In [10]:
train_loss_history = []
train_accuracy_history = []
test_loss_history = []
test_accuracy_history = []

In [11]:
for epoch in range(starting_epoch, epochs):

  train_loss, train_accuracy = train(model, optimizer, loss_function, train_loader, DEVICE)
  train_loss_history.append(train_loss)
  train_accuracy_history.append(train_accuracy)

  test_loss, test_accuracy = test(model, loss_function, test_loader, DEVICE)
  test_loss_history.append(test_loss)
  test_accuracy_history.append(test_accuracy)

  lr_scheduler.step()

  save_checkpoint(model,optimizer,epoch,"checkpoints/my_checkpoint.pth.tar")

  print(f'Epoch {epoch}, Train loss {train_loss}, Test loss {test_loss}, Train accuracy {train_accuracy}, Test accuracy {test_accuracy}')

RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[128, 64, 32, 32] to have 3 channels, but got 64 channels instead